# Prepare run

## Import libraries and functions

In [2]:
import os
import re
import keras
import scipy
import pickle
import crested
import anndata
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter
from crested.tl.zoo import dilated_cnn
from crested.tl.data import AnnDataModule
from crested.tl import default_configs, TaskConfig

import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

## Setup directories

Download the data for the notebooks from the dedicated Zenodo link of the CREsted paper. Then use it below.

In [ ]:
data_dir = Path("../../../crested_data/Figure_4/deepccl")

Output directory, to save the results from the analysis

In [ ]:
output_dir = Path(f'results')
output_path = data_dir.joinpath(output_dir)
Path(output_path).mkdir(parents=True, exist_ok=True)
Path(output_path.joinpath(Path("data"))).mkdir(parents=True, exist_ok=True)
Path(output_path.joinpath(Path("models"))).mkdir(parents=True, exist_ok=True)

Load hg38 genome

This notebook requires an hg38 fasta file and an hg38 chromosome sizes file. You can download that here for example: https://hgdownload.soe.ucsc.edu/downloads.html Once downloaded, we load them to the notebook.

In [ ]:
genome_dir = "../../../human/genome/"
genome_fasta = f"{genome_dir}hg38.fa"
genome_chrom_sizes  = f"{genome_dir}hg38.chrom.sizes"

genome = crested.Genome(genome_fasta, genome_chrom_sizes)
crested.register_genome(genome)

# Load and process data

### Import bigwigs/peaks

In [10]:
adata = crested.import_bigwigs(
    bigwigs_folder=data_dir.joinpath(Path("bigwigs")),
    regions_file=data_dir.joinpath(Path("peaks/consensus_regions.bed")),
    chromsizes_file=genome_chrom_sizes,
    target_region_width=1000,
    target="mean",
)

adata

2025-03-30T03:24:41.557570+0200 INFO Extracting values from 8 bigWig files...


AnnData object with n_obs × n_vars = 8 × 414732
    obs: 'file_path'
    var: 'chr', 'start', 'end'

### Train/Val/Test split

In [11]:
crested.pp.train_val_test_split(
    adata,
    strategy="chr",
    val_chroms=["chr7", "chr8", "chr9", "chr10"],
    test_chroms=["chr5", "chr6"]
)

print(adata.var["split"].value_counts())
adata.var.head()

split
train    285790
val       80674
test      48268
Name: count, dtype: int64


,chr,start,end,split
region,,,,
chr1:838297-838797,chr1,838297,838797,train
chr1:855502-856002,chr1,855502,856002,train
chr1:860436-860936,chr1,860436,860936,train
chr1:865601-866101,chr1,865601,866101,train
chr1:901316-901816,chr1,901316,901816,train


### Region Width

In [12]:
crested.pp.change_regions_width(
    adata,
    2114,
    chromsizes_file=genome_chrom_sizes,
)

#### Save data

In [13]:
adata.write_h5ad(output_path.joinpath(Path("data/original_data.h5ad")))

### Peak Normalization

In [14]:
regions_to_norm = crested.pp.normalize_peaks(
    adata,
    peak_threshold=0,
    gini_std_threshold=1.0,
    top_k_percent=0.03,
)

2025-03-30T03:26:41.485318+0200 INFO Filtering on top k Gini scores...
2025-03-30T03:26:42.987448+0200 INFO Added normalization weights to adata.obsm['weights']...


In [15]:
crested.pl.bar.normalization_weights(
    adata,
    title="Normalization Weights per Cell Type"
)

#### Save normalized data

In [16]:
adata.write_h5ad(output_path.joinpath(Path("data/normalized_data_003.h5ad")))

### Specificity Filtering

#### Standard Gini approach

In [17]:
adata = anndata.read_h5ad(output_path.joinpath(Path("data/normalized_data_003.h5ad")))
crested.pp.filter_regions_on_specificity(
    adata,
    gini_std_threshold=1.0,
)
adata.write_h5ad(output_path.joinpath(Path("data/filtered_data_003.h5ad")))
adata

2025-03-30T03:27:04.589110+0200 INFO After specificity filtering, kept 72365 out of 414732 regions.


AnnData object with n_obs × n_vars = 8 × 72365
    obs: 'file_path'
    var: 'chr', 'start', 'end', 'split'
    obsm: 'weights'

#### Coefficient of Variation approach

In [18]:
adata = anndata.read_h5ad(output_path.joinpath(Path("data/normalized_data_003.h5ad")))

cv = adata.X.std(axis=0)/adata.X.mean(axis=0)
q2 = np.quantile(cv, 0.5)
idx = cv > q2

adata_filtered = adata.copy()
target_matrix = adata_filtered.X.T
target_matrix_filt = target_matrix[idx]
regions_filt = adata_filtered.var_names[idx]
print(
    f"After specificity filtering, kept {len(target_matrix_filt)} out of {target_matrix.shape[0]} regions."
)
adata_filtered._inplace_subset_var(regions_filt)
adata_filtered.write_h5ad(output_path.joinpath(Path("data/filtered_data_003_cvmedian.h5ad")))
adata_filtered

After specificity filtering, kept 207366 out of 414732 regions.


AnnData object with n_obs × n_vars = 8 × 207366
    obs: 'file_path'
    var: 'chr', 'start', 'end', 'split'
    obsm: 'weights'

# Train model

In [19]:
adata = anndata.read_h5ad(output_path.joinpath(Path("data/normalized_data_003.h5ad")))

### EDA

In [20]:
for i in range(0, 8):
    print(adata.obs_names[i])
    print(f'Mean = {adata.X[i, :].mean()}')
    print(f'Max = {adata.X[i, :].max()}\n')

a172
Mean = 0.4184081256389618
Max = 20.347440719604492

gm12878
Mean = 0.32883426547050476
Max = 16.487417221069336

hepg2
Mean = 0.3343053162097931
Max = 31.6606502532959

ln229
Mean = 0.4079524874687195
Max = 14.843441009521484

m059j
Mean = 0.47035539150238037
Max = 13.188594818115234

mm001
Mean = 0.3227819502353668
Max = 28.653989791870117

mm029
Mean = 0.3212515115737915
Max = 16.878843307495117

mm099
Mean = 0.43056607246398926
Max = 17.562339782714844



### Initialize data module

In [22]:
datamodule = AnnDataModule(
    adata,
    chromsizes_file=genome_chrom_sizes,
    batch_size=512,
    max_stochastic_shift=5,
    always_reverse_complement=True,
)

### Define model and task

In [23]:
# Load DilatedCNN architecture for a dataset with 2114bp regions and 8 cell types
model = dilated_cnn(
    seq_len=2114,
    num_classes=8,
    n_dil_layers=8,
    first_conv_filters=512,
    first_conv_filter_size=11,
    filter_size=3,
    first_conv_dropout=0.2,
    dropout=0.1,
    first_conv_activation='gelu',
    activation='swish',
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence            │ (None, 2114, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 2114, 512) │     22,528 │ sequence[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 2114, 512) │      2,048 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 2114, 512) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2114, 512) │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_1conv         │ (None, 2110, 512) │    786,432 │ dropout[0][0]     │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_1bn           │ (None, 2110, 512) │      2,048 │ bpnet_1conv[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_1activation   │ (None, 2110, 512) │          0 │ bpnet_1bn[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_1crop         │ (None, 2110, 512) │          0 │ dropout[0][0]     │
│ (Cropping1D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 2110, 512) │          0 │ bpnet_1activatio… │
│                     │                   │            │ bpnet_1crop[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_1dropout      │ (None, 2110, 512) │          0 │ add[0][0]         │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_2conv         │ (None, 2102, 512) │    786,432 │ bpnet_1dropout[0… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_2bn           │ (None, 2102, 512) │      2,048 │ bpnet_2conv[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_2activation   │ (None, 2102, 512) │          0 │ bpnet_2bn[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_2crop         │ (None, 2102, 512) │          0 │ bpnet_1dropout[0… │
│ (Cropping1D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 2102, 512) │          0 │ bpnet_2activatio… │
│                     │                   │            │ bpnet_2crop[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bpnet_2dropout      │ (None, 2102, 512) │          0 │ add_1[0][0]       │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 6,336,520 (24.17 MB)

 Trainable params: 6,327,304 (24.14 MB)

 Non-trainable params: 9,216 (36.00 KB)

#### Config creation

In [24]:
loss = crested.tl.losses.PoissonLoss(log_transform=True)
optimizer = keras.optimizers.Lion(learning_rate=5e-5, weight_decay=1e-2)
metrics = [
            keras.metrics.MeanAbsoluteError(),
            keras.metrics.MeanSquaredError(),
            keras.metrics.CosineSimilarity(axis=1),
            crested.tl.metrics.PearsonCorrelation(),
            crested.tl.metrics.ConcordanceCorrelationCoefficient(),
            crested.tl.metrics.PearsonCorrelationLog(),
            crested.tl.metrics.ZeroPenaltyMetric(),
]
alternative_config = TaskConfig(optimizer, loss, metrics)
print(alternative_config)

TaskConfig(optimizer=<keras.src.optimizers.lion.Lion object at 0x1460cb77bd50>, loss=<crested.tl.losses._poisson.PoissonLoss object at 0x1460bc57af50>, metrics=[<MeanAbsoluteError name=mean_absolute_error>, <MeanSquaredError name=mean_squared_error>, <CosineSimilarity name=cosine_similarity>, <PearsonCorrelation name=pearson_correlation>, <ConcordanceCorrelationCoefficient name=concordance_correlation_coefficient>, <PearsonCorrelationLog name=pearson_correlation_log>, <ZeroPenaltyMetric name=zero_penalty_metric>])


### Training

In [26]:
# setup the trainer
trainer = crested.tl.Crested(
    data=datamodule,
    model=model,
    config=alternative_config,
    project_name="deepccl_training",
    run_name="deepccl_base",
    logger="wandb",
    seed=42,
)

In [ ]:
# train the model
trainer.fit(
    epochs=100,
    model_checkpointing_best_only=False
    save_dir=str(output_path.joinpath(Path("deepccl_training")))
)

# Finetune model

## Load model and data

In [30]:
# Best base model
model = keras.models.load_model(
    output_path.joinpath(Path("models/DeepCCL_base.keras")),
    compile=True,
)

# Filtered regions based on coefficient of variation
adata = anndata.read_h5ad(output_path.joinpath(Path("data/filtered_data_003_cvmedian.h5ad")))
datamodule = AnnDataModule(
    adata,
    chromsizes_file=genome_chrom_sizes,
    batch_size=64,
    max_stochastic_shift=5,
    always_reverse_complement=True,
)

adata

AnnData object with n_obs × n_vars = 8 × 207366
    obs: 'file_path'
    var: 'chr', 'start', 'end', 'split'
    obsm: 'weights'

## Config creation

In [31]:
# Same loss/optimizer with a reduced learning rate
loss = crested.tl.losses.PoissonLoss(log_transform=True)
optimizer = keras.optimizers.Lion(learning_rate=5e-6, weight_decay=1e-1)
metrics = [
            keras.metrics.MeanAbsoluteError(),
            keras.metrics.MeanSquaredError(),
            keras.metrics.CosineSimilarity(axis=1),
            crested.tl.metrics.PearsonCorrelation(),
            crested.tl.metrics.ConcordanceCorrelationCoefficient(),
            crested.tl.metrics.PearsonCorrelationLog(),
            crested.tl.metrics.ZeroPenaltyMetric(),
]
alternative_config = TaskConfig(optimizer, loss, metrics)

## Finetuning

In [32]:
trainer = crested.tl.Crested(
    data=datamodule,
    model=model,
    config=alternative_config,
    project_name="deepccl_training",
    run_name="deepccl",
    logger="wandb",
    seed=42,
)

In [ ]:
trainer.fit(
    epochs=60,
    model_checkpointing_best_only=False,
    save_dir=str(output_path.joinpath(Path("deepccl_training")))
)